# Análise de Erros e Matriz de Confusão 🔍

Neste notebook, aprofundamos a avaliação do modelo treinado, identificando Falsos Positivos e Falsos Negativos.

In [ ]:
import torch
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import sys
import os

sys.path.append(os.path.abspath(".."))
from models.models import CharCNN
from utils.utils import url_to_tensor, VOCAB_SIZE, decode_tensor

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# 1. Carregar Modelo Treinado (CNN Exemplo)
MODEL_PATH = "../checkpoints/urlifeguard_cnn.pth"
model = CharCNN(vocab_size=VOCAB_SIZE, num_classes=2).to(device)

if os.path.exists(MODEL_PATH):
    model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
    print("Modelo carregado com sucesso!")
else:
    print("Modelo não encontrado. Treine primeiro!")

In [ ]:
# 2. Carregar Dados de Teste
df = pd.read_csv("../data/malicious_phish.csv")
df['label'] = df['type'].apply(lambda x: 0 if x == 'benign' else 1)

# Usar uma amostra para análise rápida
test_sample = df.sample(1000, random_state=99)
urls = test_sample['url'].values
y_true = test_sample['label'].values

In [ ]:
# 3. Gerar Predições
model.eval()
y_pred = []

with torch.no_grad():
    for url in urls:
        tensor = url_to_tensor(url, max_len=150).unsqueeze(0).to(device)
        output = model(tensor)
        _, predicted = torch.max(output, 1)
        y_pred.append(predicted.item())

In [ ]:
# 4. Matriz de Confusão e Relatório
print(classification_report(y_true, y_pred, target_names=['Benigno', 'Malicioso']))

cm = confusion_matrix(y_true, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Benigno', 'Malicioso'])
disp.plot(cmap='Blues')
plt.title("Matriz de Confusão")
plt.show()

In [ ]:
# 5. Análise de Falhas (Onde o modelo errou?)
test_sample['pred'] = y_pred
errors = test_sample[test_sample['label'] != test_sample['pred']]

print(f"Total de erros na amostra: {len(errors)}")
print("\nExemplos de Falsos Negativos (Era Malicioso, disse Benigno):")
fn = errors[(errors['label'] == 1) & (errors['pred'] == 0)]
display(fn[['url', 'type']].head(10))

print("\nExemplos de Falsos Positivos (Era Benigno, disse Malicioso):")
fp = errors[(errors['label'] == 0) & (errors['pred'] == 1)]
display(fp[['url', 'type']].head(10))